**1) Load in all the needed packages**

In [0]:
import os
import re
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import tensorflow.python.platform
import matplotlib.pyplot as plt
import zipfile
import pickle
import random

from tensorflow.python.platform import gfile
from tensorflow.contrib import learn
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC, LinearSVC
from IPython.core.display import Image, display


**2) unzip the needed dataset**

In [2]:
print('TensorFlow Verions: %s' % tf.__version__)
local_zip = '/content/Chess_ID.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

TensorFlow Verions: 1.13.1


In [0]:
#Create a list of all the image locations for loading
model_dir = '/content/'
images_dir = '/content/Chess_ID/train/'
CATEGORIES = ["bb", "bk", "bn", "bp", "bq", "br", "empty", "wb", "wk", "wn", "wp", "wq", "wr"]
final_list =[]
for category in CATEGORIES:
  list_images = [images_dir+category+"/"+f for f in os.listdir(images_dir+category) if re.search('jpg|JPG', f)]
  final_list = final_list + list_images
list_images = final_list
len_images = len(list_images)
#shuffle as the images will be ordered
random.shuffle(list_images)

In [0]:
#Load in Inception
def create_graph():
  #loading in classify_image_graph_def.pb
  with gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [5]:
#extract the deep features in the last layer of the inception model. 
#takes in list_images, 
nb_features = 2048 #number of deep features per image
features = np.empty((len_images , nb_features)) # Deep Features to be stored here
labels = [] # Image class labels to be stored here
create_graph() # Create a TensorFlow computation graph

with tf.Session() as s:
  #grabbing the 
  almost_last_inception_layer = s.graph.get_tensor_by_name('pool_3:0') 
  print(enumerate(list_images))
  for ind, image in enumerate(list_images):
    if (ind%10 == 0):
      print('Processing %s ...' % (image))
    if not gfile.Exists(image):
      tf.logging.fatal('File does not exist %s', image)
    #Read in the image
    image_data = gfile.FastGFile(image, 'rb').read()
    predictions = s.run(almost_last_inception_layer,
                           {'DecodeJpeg/contents:0': image_data})
    features[ind,:] = np.squeeze(predictions)
    #label images by the folder they are in 
    labels.append(re.split('_\d+',image.split('/')[4])[0])
    
  print('')
  print('Done!')

Instructions for updating:
Use tf.gfile.GFile.
Processing /content/Chess_ID/train/empty/empty.1810.jpg ...
Processing /content/Chess_ID/train/bb/bb.50.jpg ...
Processing /content/Chess_ID/train/bp/bp.129.jpg ...
Processing /content/Chess_ID/train/empty/empty.112.jpg ...
Processing /content/Chess_ID/train/bn/bn.239.jpg ...
Processing /content/Chess_ID/train/bp/bp.1365.jpg ...
Processing /content/Chess_ID/train/bb/bb.412.jpg ...
Processing /content/Chess_ID/train/empty/empty.229.jpg ...
Processing /content/Chess_ID/train/empty/empty.2673.jpg ...
Processing /content/Chess_ID/train/bq/bq.149.jpg ...
Processing /content/Chess_ID/train/empty/empty.3081.jpg ...
Processing /content/Chess_ID/train/wp/wp.36.jpg ...
Processing /content/Chess_ID/train/bb/bb.228.jpg ...
Processing /content/Chess_ID/train/empty/empty.817.jpg ...
Processing /content/Chess_ID/train/bb/bb.425.jpg ...
Processing /content/Chess_ID/train/bp/bp.755.jpg ...
Processing /content/Chess_ID/train/wp/wp.1103.jpg ...
Processing /c

In [0]:
labels_number = []
#assign labels by index in CATEGORIES
for i in range(len(labels)):
  indexed = CATEGORIES.index(labels[i])
  labels_number.append(indexed)
#pickle it for later use to save time
pickle.dump(features, open('features', 'wb'))
pickle.dump(labels_number, open('labels_number', 'wb'))
pickle.dump(labels, open('labels', 'wb'))

**unpickle here if needed, you can start here when grading if you want, processing takes about 30-40 mins I think **

In [0]:
#Load in data
features       = pickle.load(open('features', 'rb'))
labels_number  = pickle.load(open('labels_number', 'rb'))
labels         = pickle.load(open('labels', 'rb'))


In [40]:
#assign 75% of images as training data and the other 25% as test data
image_features_train, image_features_test, image_label_train, image_label_test = sklearn.model_selection.train_test_split(features, labels_number, test_size=0.25, random_state=1)
n_classes = len(set(image_label_train))
dimension=image_features_train.shape[1]
print(dimension)

2048


In [0]:

#create our feature columns
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=image_features_train.shape[1])]
#apply a Linear Classifier to learn, and save the model with it's checkpoints
estimator = learn.LinearClassifier(feature_columns = 
                        feature_columns, n_classes=n_classes, model_dir="final-model")
#Fit the model
estimator.fit(image_features_train, image_label_train, steps=1000)
#Print out the predicted labels
image_label_predicted = list(estimator.predict(image_features_test))

print('Done!')



In [0]:
print("Accuracy: {0:0.2f}%".format(accuracy_score(image_label_test, image_label_predicted)*100))

At this point, we should have out model saved as Final model which we can export and use in our final project.

You can see below my attempts at a more efficient save and predict, 
but ultimately I was running into bugs no one seemed to have answers to and gave up



In [0]:
# from tensorflow.contrib.layers import create_feature_spec_for_parsing
# from tensorflow.contrib.learn.python.learn.utils import input_fn_utils

# def save_tf_learn_model(estimator, feature_columns, export_dir):
#     feature_spec = create_feature_spec_for_parsing(feature_columns)
#     serving_input_fn = input_fn_utils.build_parsing_serving_input_fn(feature_spec)
#     estimator.export_savedmodel(export_dir, serving_input_fn)
#     print("Done exporting tf.learn model to " + export_dir + "!")


In [0]:
# SESS_DICT = {}
# def get_session(model_id):
#     global SESS_DICT
#     config = tf.ConfigProto(allow_soft_placement=True)
#     SESS_DICT[model_id] = tf.Session(config=config)
#     return SESS_DICT[model_id]

# def load_tf_model(model_path):
#     sess = get_session(model_path)
#     tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], model_path)
#     return sess

In [0]:
# load_tf_model("/content/night_chess_model/1557453716")
# print("done")

In [0]:
# from tensorflow.contrib import predictor
# # test_the_feature = np.squeeze(test_feature)
# predict_fn = predictor.from_saved_model("/content/night_chess_model/1557453716")
# predictions = predict_fn(
#     {"inputs": test_feature})
# print(predictions['scores'])

In [0]:

###current test
# def serving_input_receiver_fn():
#   """Build the serving inputs."""
#   # The outer dimension (None) allows us to batch up inputs for
#   # efficiency. However, it also means that if we want a prediction
#   # for a single instance, we'll need to wrap it in an outer list.
#   inputs = {"x": tf.placeholder(shape=[None, 4], dtype=tf.float32)}
#   return tf.estimator.export.ServingInputReceiver(inputs, inputs)

# export_dir = estimator.export_savedmodel(
# export_dir_base="test_model",
# serving_input_fn = serving_input_receiver_fn)